# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [53]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

In [54]:
!pip install yelpapi
!pip install tqdm

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [55]:
# Load API Credentials
with open('yelp_api.json', 'r') as f:
    login = json.load(f)

In [56]:
login.keys()

dict_keys(['client-id', 'api-key'])

In [57]:
#login.items()
#login['API Key']

In [58]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['api-key'], timeout_s = 5.0)

### Define Search Terms and File Paths

In [59]:
# set our API call parameters and filename before the first call
location = 'Seattle, WA 98122'
term = 'pizza'

In [60]:
location.split(',')[0]

'Seattle'

In [61]:
## Specify fodler for saving data
FOLDER = 'Data/'

os.makedirs(FOLDER, exist_ok = True)
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [13]:
JSON_FILE

'Data/Seattle-pizza.json'

### Check if Json File exists and Create it if it doesn't

In [62]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
          json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Seattle-pizza.json already exists.


### Load JSON FIle and account for previous results

### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [63]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term = term, location = location)

In [64]:
type(results)

dict

In [65]:
len(results)

3

In [66]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [67]:
results['total']

855

In [68]:
results['region']

{'center': {'longitude': -122.29568481445312, 'latitude': 47.61309534151991}}

In [69]:
results['businesses']

[{'id': 'sx8OeNntGUg3BzQV_L8z6g',
  'alias': 'jackson-street-pizza-lounge-seattle',
  'name': 'Jackson Street Pizza Lounge',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/X9EeTjVjULdmyW9XMowPYA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/jackson-street-pizza-lounge-seattle?adjust_creative=UDy0-CMNxI6EjoRoxUg7Iw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=UDy0-CMNxI6EjoRoxUg7Iw',
  'review_count': 17,
  'categories': [{'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
   {'alias': 'pizza', 'title': 'Pizza'}],
  'rating': 4.0,
  'coordinates': {'latitude': 47.59916307243915,
   'longitude': -122.2948217963242},
  'transactions': [],
  'location': {'address1': '2901 S Jackson St',
   'address2': '',
   'address3': None,
   'city': 'Seattle',
   'zip_code': '98144',
   'country': 'US',
   'state': 'WA',
   'display_address': ['2901 S Jackson St', 'Seattle, WA 98144']},
  'phone': '+12064857118',
  'display_phone': '(206) 485-7118',

In [70]:
## How many results total?
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,sx8OeNntGUg3BzQV_L8z6g,jackson-street-pizza-lounge-seattle,Jackson Street Pizza Lounge,https://s3-media3.fl.yelpcdn.com/bphoto/X9EeTj...,False,https://www.yelp.com/biz/jackson-street-pizza-...,17,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.0,"{'latitude': 47.59916307243915, 'longitude': -...",[],"{'address1': '2901 S Jackson St', 'address2': ...",+12064857118,(206) 485-7118,1550.546409,NaN
1,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,143,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}",[pickup],"{'address1': '601 Summit Ave E', 'address2': '...",,,2578.701393,$$
2,ENxs0vvtkJYxQqaZ3sbEjw,my-friend-dereks-seattle,My Friend Derek's,https://s3-media1.fl.yelpcdn.com/bphoto/MVV6S3...,False,https://www.yelp.com/biz/my-friend-dereks-seat...,34,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",5.0,"{'latitude': 47.651515, 'longitude': -122.343613}",[],"{'address1': '3632 Woodland Park Ave N', 'addr...",,,5581.310444,NaN
3,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media1.fl.yelpcdn.com/bphoto/UNfR-J...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,370,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","[pickup, delivery]","{'address1': '1546 15th Ave', 'address2': '', ...",+12068388081,(206) 838-8081,1292.526796,$$
4,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,66,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.61849, 'longitude': -122.31664}",[],"{'address1': '1830 12th Ave', 'address2': '', ...",+12064031809,(206) 403-1809,1690.356563,NaN
5,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,379,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",[delivery],"{'address1': '525 Rainier Ave S', 'address2': ...",+12063295133,(206) 329-5133,2166.834011,$$
6,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,983,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...","[pickup, delivery]","{'address1': '700 E Pine St', 'address2': '', ...",+12063226444,(206) 322-6444,2080.359405,$
7,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,183,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.636044, 'longitude': -122.277377}",[delivery],"{'address1': '4235 E Madison St', 'address2': ...",+12068606110,(206) 860-6110,2897.252844,$$
8,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/-aZrUu...,False,https://www.yelp.com/biz/italian-family-pizza-...,1007,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.60937, 'longitude': -122.32546}","[pickup, delivery]","{'address1': '1028 Madison St', 'address2': No...",+12065380040,(206) 538-0040,2271.307105,$$
9,ZbpJ1mPQfGJRkSiG4eHzdQ,alexandras-macarons-seattle-2,Alexandras Macarons,https://s3-media4.fl.yelpcdn.com/bphoto/fSkfjz...,False,https://www.yelp.com/biz/alexandras-macarons-s...,76,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.5,"{'latitude': 47.6132799970069, 'longitude': -1...","[pickup, delivery]","{'address1': '1410 18th Ave', 'address2': '', ...",+12065183895,(206) 518-3895,969.319176,$


- Where is the actual data we want to save?

In [71]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [72]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total'])/ results_per_page)
n_pages

43

In [74]:
for i in tqdm_notebook(range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        
        time.sleep(.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(location = location, term = term,
                                   offset = n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])
        
        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.
    except Exception as e:
        print(' [!] ERROR', e)


  0%|          | 0/43 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [28]:
df = pd.read_json(JSON_FILE)

In [29]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,355,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","[pickup, delivery]",$$,"{'address1': '1546 15th Ave', 'address2': '', ...",+12068388081,(206) 838-8081,1292.526796
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,369,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",[delivery],$$,"{'address1': '525 Rainier Ave S', 'address2': ...",+12063295133,(206) 329-5133,2166.834011
2,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],NaN,"{'address1': '2300 E Cherry St', 'address2': '...",,,749.173113
3,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,969,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...","[pickup, delivery]",$,"{'address1': '700 E Pine St', 'address2': '', ...",+12063226444,(206) 322-6444,2080.359405
4,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/-aZrUu...,False,https://www.yelp.com/biz/italian-family-pizza-...,1009,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.60937, 'longitude': -122.32546}","[pickup, delivery]",$$,"{'address1': '1028 Madison St', 'address2': No...",+12065380040,(206) 538-0040,2271.307105


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 839 entries, 0 to 838
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             839 non-null    object 
 1   alias          839 non-null    object 
 2   name           839 non-null    object 
 3   image_url      839 non-null    object 
 4   is_closed      839 non-null    bool   
 5   url            839 non-null    object 
 6   review_count   839 non-null    int64  
 7   categories     839 non-null    object 
 8   rating         839 non-null    float64
 9   coordinates    839 non-null    object 
 10  transactions   839 non-null    object 
 11  price          740 non-null    object 
 12  location       839 non-null    object 
 13  phone          839 non-null    object 
 14  display_phone  839 non-null    object 
 15  distance       839 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 99.3+ KB


In [31]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Seattle-pizza.csv.gz'

In [32]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression = 'gzip', index = False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [33]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 819,555 Bytes
CSV.GZ FILE: 117,305 Bytes
the csv.gz is 6.986530838412685 times smaller!


## Next Class: Processing the Results and Mapping 